In [2]:
import keras
import nltk
nltk.download('all')
import pickle
import json
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
import random
import datetime
from googlesearch import *
import webbrowser
import requests
from pycricbuzz import Cricbuzz
import billboard
import time
from pygame import mixer
import COVID19Py

from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

words=[]
classes=[]
documents=[]
ignore=['?','!',',',"'s"]

data_file=open('intents.json').read()
intents=json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w=nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w,intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
words=[lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore]
words=sorted(list(set(words)))
classes=sorted(list(set(classes)))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

#training data
training=[]
output_empty=[0]*len(classes)

for doc in documents:
    bag=[]
    pattern=doc[0]
    pattern=[ lemmatizer.lemmatize(word.lower()) for word in pattern ]
    
    for word in words:
        if word in pattern:
            bag.append(1)
        else:
            bag.append(0)
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    
    training.append([bag,output_row])
    
random.shuffle(training)
training=np.array(training)  
X_train=list(training[:,0])
y_train=list(training[:,1])  

from tensorflow.keras.optimizers import Adam

#Model
model=Sequential()
model.add(Dense(128,activation='relu',input_shape=(len(X_train[0]),)))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]),activation='softmax'))

adam=Adam(0.001)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
#model.fit(np.array(X_train),np.array(y_train),epochs=200,batch_size=10,verbose=1)
weights=model.fit(np.array(X_train),np.array(y_train),epochs=500,batch_size=10,verbose=1)    
model.save('mymodel.h5',weights)


Epoch 1/500


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]    |   Package basqu

12/12 [==============================] - 0s 2ms/step - loss: 3.4371 - accuracy: 0.0000e+00
Epoch 2/500
12/12 [==============================] - 0s 1ms/step - loss: 3.3951 - accuracy: 0.0345
Epoch 3/500
12/12 [==============================] - 0s 2ms/step - loss: 3.3885 - accuracy: 0.0259
Epoch 4/500
12/12 [==============================] - 0s 2ms/step - loss: 3.3448 - accuracy: 0.0862
Epoch 5/500
12/12 [==============================] - 0s 3ms/step - loss: 3.3488 - accuracy: 0.0862
Epoch 6/500
12/12 [==============================] - 0s 2ms/step - loss: 3.3156 - accuracy: 0.1034
Epoch 7/500
12/12 [==============================] - 0s 2ms/step - loss: 3.2648 - accuracy: 0.1466
Epoch 8/500
12/12 [==============================] - 0s 2ms/step - loss: 3.2119 - accuracy: 0.1379
Epoch 9/500
12/12 [==============================] - 0s 2ms/step - loss: 3.1757 - accuracy: 0.1638
Epoch 10/500
12/12 [==============================] - 0s 2ms/step - loss: 3.1700 - accuracy: 0.1552
Epoch 11/500
12/1

12/12 [==============================] - 0s 1ms/step - loss: 0.3452 - accuracy: 0.9224
Epoch 84/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4161 - accuracy: 0.8879
Epoch 85/500
12/12 [==============================] - 0s 2ms/step - loss: 0.3366 - accuracy: 0.9052
Epoch 86/500
12/12 [==============================] - 0s 2ms/step - loss: 0.2634 - accuracy: 0.9224
Epoch 87/500
12/12 [==============================] - 0s 2ms/step - loss: 0.3875 - accuracy: 0.9052
Epoch 88/500
12/12 [==============================] - 0s 1ms/step - loss: 0.3923 - accuracy: 0.9138
Epoch 89/500
12/12 [==============================] - 0s 2ms/step - loss: 0.3805 - accuracy: 0.8966
Epoch 90/500
12/12 [==============================] - 0s 1ms/step - loss: 0.3879 - accuracy: 0.9138
Epoch 91/500
12/12 [==============================] - 0s 2ms/step - loss: 0.2970 - accuracy: 0.9052
Epoch 92/500
12/12 [==============================] - 0s 2ms/step - loss: 0.2524 - accuracy: 0.9310
Epoch 93/500


12/12 [==============================] - 0s 2ms/step - loss: 0.1641 - accuracy: 0.9483
Epoch 165/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1637 - accuracy: 0.9569
Epoch 166/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1164 - accuracy: 0.9741
Epoch 167/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0854 - accuracy: 0.9828
Epoch 168/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1555 - accuracy: 0.9569
Epoch 169/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1265 - accuracy: 0.9741
Epoch 170/500
12/12 [==============================] - 0s 3ms/step - loss: 0.1271 - accuracy: 0.9828
Epoch 171/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1051 - accuracy: 0.9569
Epoch 172/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1282 - accuracy: 0.9569
Epoch 173/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1972 - accuracy: 0.9483
Epoc

12/12 [==============================] - 0s 2ms/step - loss: 0.2343 - accuracy: 0.9052
Epoch 246/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0763 - accuracy: 0.9828
Epoch 247/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0969 - accuracy: 0.9741
Epoch 248/500
12/12 [==============================] - 0s 1ms/step - loss: 0.1013 - accuracy: 0.9741
Epoch 249/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0941 - accuracy: 0.9741
Epoch 250/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0948 - accuracy: 0.9741
Epoch 251/500
12/12 [==============================] - 0s 3ms/step - loss: 0.1025 - accuracy: 0.9569
Epoch 252/500
12/12 [==============================] - 0s 1ms/step - loss: 0.1474 - accuracy: 0.9310
Epoch 253/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0963 - accuracy: 0.9741
Epoch 254/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1212 - accuracy: 0.9397
Epoc

12/12 [==============================] - 0s 2ms/step - loss: 0.0603 - accuracy: 0.9828
Epoch 327/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0811 - accuracy: 0.9569
Epoch 328/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0612 - accuracy: 0.9828
Epoch 329/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0562 - accuracy: 0.9741
Epoch 330/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0922 - accuracy: 0.9655
Epoch 331/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0800 - accuracy: 0.9569
Epoch 332/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0703 - accuracy: 0.9828
Epoch 333/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0744 - accuracy: 0.9828
Epoch 334/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0668 - accuracy: 0.9828
Epoch 335/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0780 - accuracy: 0.9828
Epoc

12/12 [==============================] - 0s 2ms/step - loss: 0.0245 - accuracy: 0.9914
Epoch 408/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0480 - accuracy: 0.9655
Epoch 409/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0579 - accuracy: 0.9914
Epoch 410/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0605 - accuracy: 0.9828
Epoch 411/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0589 - accuracy: 0.9655
Epoch 412/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0683 - accuracy: 0.9655
Epoch 413/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0353 - accuracy: 0.9828
Epoch 414/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0954 - accuracy: 0.9569
Epoch 415/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0384 - accuracy: 0.9828
Epoch 416/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0594 - accuracy: 0.9828
Epoc

12/12 [==============================] - 0s 2ms/step - loss: 0.0407 - accuracy: 0.9655
Epoch 489/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0547 - accuracy: 0.9741
Epoch 490/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0247 - accuracy: 1.0000
Epoch 491/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0240 - accuracy: 0.9914
Epoch 492/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0243 - accuracy: 0.9914
Epoch 493/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0685 - accuracy: 0.9655
Epoch 494/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0452 - accuracy: 0.9828
Epoch 495/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0628 - accuracy: 0.9655
Epoch 496/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0444 - accuracy: 0.9828
Epoch 497/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0681 - accuracy: 0.9655
Epoc

In [ ]:

from keras.models import load_model
model = load_model('mymodel.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


#Predict
def clean_up(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[ lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def create_bow(sentence,words):
    sentence_words=clean_up(sentence)
    bag=list(np.zeros(len(words)))
    
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence,model):
    p=create_bow(sentence,words)
    res=model.predict(np.array([p]))[0]
    threshold=0.8
    results=[[i,r] for i,r in enumerate(res) if r>threshold]
    results.sort(key=lambda x: x[1],reverse=True)
    return_list=[]
    
    for result in results:
        return_list.append({'intent':classes[result[0]],'prob':str(result[1])})
    return return_list

def get_response(return_list,intents_json):
    
    if len(return_list)==0:
        tag='noanswer'
    else:    
        tag=return_list[0]['intent']
    if tag=='datetime':        
        print(time.strftime("%A"))
        print (time.strftime("%d %B %Y"))
        print (time.strftime("%H:%M:%S"))

    if tag=='google':
        query=input('Enter query...')
        chrome_path = r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe %s'
        for url in search(query, tld="co.in", num=1, stop = 1, pause = 2):
            webbrowser.open("https://google.com/search?q=%s" % query)
    if tag=='weather':
        api_key='987f44e8c16780be8c85e25a409ed07b'
        base_url = "http://api.openweathermap.org/data/2.5/weather?"
        city_name = input("Enter city name : ")
        complete_url = base_url + "appid=" + api_key + "&q=" + city_name
        response = requests.get(complete_url) 
        x=response.json()
        print('Present temp.: ',round(x['main']['temp']-273,2),'celcius ')
        print('Feels Like:: ',round(x['main']['feels_like']-273,2),'celcius ')
        print(x['weather'][0]['main'])
        
    if tag=='news':
        main_url = " http://newsapi.org/v2/top-headlines?country=in&apiKey=bc88c2e1ddd440d1be2cb0788d027ae2"
        open_news_page = requests.get(main_url).json()
        article = open_news_page["articles"]
        results = [] 
          
        for ar in article: 
            results.append([ar["title"],ar["url"]]) 
          
        for i in range(10): 
            print(i + 1, results[i][0])
            print(results[i][1],'\n')
            
    if tag=='cricket':
        c = Cricbuzz()
        matches = c.matches()
        for match in matches:
            print(match['srs'],' ',match['mnum'],' ',match['status'])
    
    if tag=='song':
        chart=billboard.ChartData('hot-100')
        print('The top 10 songs at the moment are:')
        for i in range(10):
            song=chart[i]
            print(song.title,'- ',song.artist)
    if tag=='timer':        
        mixer.init()
        x=input('Minutes to timer..')
        time.sleep(float(x)*60)
        mixer.music.load('Handbell-ringing-sound-effect.mp3')
        mixer.music.play()
        
    if tag=='covid19':
        
        covid19=COVID19Py.COVID19(data_source='jhu')
        country=input('Enter Location...')
        
        if country.lower()=='world':
            latest_world=covid19.getLatest()
            print('Confirmed:',latest_world['confirmed'],' Deaths:',latest_world['deaths'])
        
        else:
                
            latest=covid19.getLocations()
            
            latest_conf=[]
            latest_deaths=[]
            for i in range(len(latest)):
                
                if latest[i]['country'].lower()== country.lower():
                    latest_conf.append(latest[i]['latest']['confirmed'])
                    latest_deaths.append(latest[i]['latest']['deaths'])
            latest_conf=np.array(latest_conf)
            latest_deaths=np.array(latest_deaths)
            print('Confirmed: ',np.sum(latest_conf),'Deaths: ',np.sum(latest_deaths))

    list_of_intents= intents_json['intents']    
    for i in list_of_intents:
        if tag==i['tag'] :
            result= random.choice(i['responses'])
    return result

def response(text):
    return_list=predict_class(text,model)
    response=get_response(return_list,intents)
    return response

while(1):
    x=input()
    print(response(x))
    if x.lower() in ['bye','goodbye','get lost','see you']:  
        break


#Self learning
print('Help me Learn?')
tag=input('Please enter general category of your question  ')
flag=-1
for i in range(len(intents['intents'])):
    if tag.lower() in intents['intents'][i]['tag']:
        intents['intents'][i]['patterns'].append(input('Enter your message: '))
        intents['intents'][i]['responses'].append(input('Enter expected reply: '))        
        flag=1

if flag==-1:
    
    intents['intents'].append (
        {'tag':tag,
         'patterns': [input('Please enter your message')],
         'responses': [input('Enter expected reply')]})
    
with open('intents.json','w') as outfile:
    outfile.write(json.dumps(intents,indent=4))

In [5]:
import requests

# Define the URL for the COVID-19 data source
url = "https://api.covid19api.com/summary"

try:
    # Send a GET request to the API
    response = requests.get(url)

    # Check if the request was successful (HTTP status code 200)
    if response.status_code == 200:
        # Parse the JSON data
        data = response.json()

        # Extract worldwide statistics
        global_stats = data["Global"]

        # Display worldwide COVID-19 statistics
        print("Worldwide COVID-19 Statistics:")
        print(f"Total Cases: {global_stats['TotalConfirmed']}")
        print(f"Total Deaths: {global_stats['TotalDeaths']}")
        print(f"Total Recovered: {global_stats['TotalRecovered']}")
    else:
        print("Failed to fetch COVID-19 data.")

except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: HTTPSConnectionPool(host='api.covid19api.com', port=443): Max retries exceeded with url: /summary (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000184F7353220>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


In [14]:
import requests

# Define the base URL for the disease.sh API
base_url = "https://disease.sh/v3/covid-19"

try:
    # Fetch global COVID-19 data
    global_data = requests.get(f"{base_url}/all").json()

    # Fetch country-specific COVID-19 data (e.g., India)
    country_data = requests.get(f"{base_url}/countries/India").json()

    # Display global COVID-19 data in key-value format
    print("Global COVID-19 Data:")
    for key, value in global_data.items():
        print(f"{key}: {value}")

    # Display country-specific COVID-19 data in key-value format
    print("\nCOVID-19 Data for India:")
    for key, value in country_data.items():
        print(f"{key}: {value}")

except Exception as e:
    print(f"An error occurred: {e}")


Global COVID-19 Data:
updated: 1694082651690
cases: 695034270
todayCases: 1
deaths: 6913063
todayDeaths: 0
recovered: 666842221
todayRecovered: 43425
active: 21278986
critical: 37788
casesPerOneMillion: 89166
deathsPerOneMillion: 886.9
tests: 7005316803
testsPerOneMillion: 881733.67
population: 7944935131
oneCasePerPeople: 0
oneDeathPerPeople: 0
oneTestPerPeople: 0
activePerOneMillion: 2678.31
recoveredPerOneMillion: 83933
criticalPerOneMillion: 4.76
affectedCountries: 231

COVID-19 Data for India:
updated: 1694083252081
country: India
countryInfo: {'_id': 356, 'iso2': 'IN', 'iso3': 'IND', 'lat': 20, 'long': 77, 'flag': 'https://disease.sh/assets/img/flags/in.png'}
cases: 44997466
todayCases: 0
deaths: 531930
todayDeaths: 0
recovered: 44464951
todayRecovered: 0
active: 585
critical: 0
casesPerOneMillion: 31990
deathsPerOneMillion: 378
tests: 930797975
testsPerOneMillion: 661721
population: 1406631776
continent: Asia
oneCasePerPeople: 31
oneDeathPerPeople: 2644
oneTestPerPeople: 2
activ

In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set your Spotify API credentials
client_id = '7262d1a4a55a46beb973fa784b78503f'
client_secret = '9d3382e5e1ae483baaf676178143cc17'

# Initialize Spotipy with your credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Search for the song "Single Soon" by Selena Gomez
results = sp.search(q='Primera Cita by Carin Leon', type='track')

# Extract Spotify links if available
if results['tracks']['total'] > 0:
    track = results['tracks']['items'][0]
    spotify_link = track['external_urls']['spotify']
    print(f"Spotify Link: {spotify_link}")
else:
    print("Song not found on Spotify.")


Spotify Link: https://open.spotify.com/track/4mGrWfDISjNjgeQnH1B8IE


In [23]:
from bs4 import BeautifulSoup
import requests

urls = ["https://www.cricbuzz.com/cricket-match/live-scores", "https://www.cricbuzz.com/cricket-match/live-scores/recent-matches", "https://www.cricbuzz.com/cricket-match/live-scores/upcoming-matches"]
x = "<ul style='margin-left: 20px;'>"
for url in urls:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        index = 0
        
        type = url.split('/')[-1]
        print("type: ->", type)
        if type == "live-scores":
            type="Live Matches"
        elif type == 'recent-matches':
            type='Recent Matches'
        elif type == 'upcoming-matches':
            type='Upcoming Matches'
        x+=f"<li>{type}</li>"
        x += "<ol style='margin-left: 30px;'>"
        
        # Find the live match updates
        live_matches = soup.find_all('div', class_='cb-mtch-lst cb-col cb-col-100 cb-tms-itm')

        if not live_matches:
            print("No live matches found.")

        for match in live_matches:
            srs = match.find('a', class_='text-hvr-underline text-bold').text.strip()
            mnum = match.find('span', class_='text-gray').text.strip()
            try:
                status = match.find('div', class_='cb-text-complete').text.strip()
            except:
                try:
                    status = match.find('div', class_='cb-text-live').text.strip()
                except:
                    status = "Upcoming Match"

            x += f"<li> {srs} - {mnum} : {status} </li>"
        x+="</ol>"
        x+="</li>"
    else:
        print("Failed to fetch cricket updates")
x+="</ul>"

print(x)

type: -> live-scores
type: -> recent-matches
type: -> upcoming-matches
<ul style='margin-left: 20px;'><li>Live Matches</li><ol style='margin-left: 30px;'><li> Japan vs Mongolia, - 6th T20I : Japan won by 158 runs </li><li> Japan vs Mongolia, - 5th T20I : Japan won by 180 runs </li><li> Belgium vs Malta, - 5th Match : Upcoming Match </li><li> Belgium vs Malta, - 4th Match : Belgium won by 16 runs </li><li> Sussex vs Glamorgan, - 19th Match : Day 2: 1st Session - Glamorgan trail by 228 runs </li><li> Gloucestershire vs Northamptonshire, - 20th Match : Day 2: 1st Session </li><li> Worcestershire vs Kent, - 23rd Match : Day 2: 1st Session </li><li> Lancashire vs Nottinghamshire, - 24th Match : Day 2: 1st Session - Nottinghamshire trail by 330 runs </li><li> Hampshire vs Durham, - 22nd Match : Day 2: 1st Session </li><li> Warwickshire vs Surrey, - 25th Match : Day 2: 1st Session </li></ol></li><li>Recent Matches</li><ol style='margin-left: 30px;'><li> Bangladesh vs Zimbabwe, - 4th T20I : Ba